In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [5]:
df1['target']=df1['target'].map({1:0,0:1})

In [6]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2'],axis=1)

In [7]:
df1.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,123472,117906,117908,0,117961118300
1,17183,1540,123125,118536,118539,0,117961118343
2,36724,14457,117884,267952,117880,0,118219118220
3,36135,5396,119993,240983,118322,0,117961118343
4,42680,5905,119569,123932,119325,0,117929117930


# Combinations with brute force 


In [8]:
from itertools import combinations

cols=list(df1.columns)
cols.remove('target')
d=df1.copy()

for j in combinations(cols,2):
    d[j[0]+'_'+j[1]]=df1[j[0]]+df1[j[1]]
    
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 22 columns):
RESOURCE                          32769 non-null object
MGR_ID                            32769 non-null object
ROLE_DEPTNAME                     32769 non-null object
ROLE_FAMILY_DESC                  32769 non-null object
ROLE_CODE                         32769 non-null object
target                            32769 non-null int64
ROLLUP_12                         32769 non-null object
RESOURCE_MGR_ID                   32769 non-null object
RESOURCE_ROLE_DEPTNAME            32769 non-null object
RESOURCE_ROLE_FAMILY_DESC         32769 non-null object
RESOURCE_ROLE_CODE                32769 non-null object
RESOURCE_ROLLUP_12                32769 non-null object
MGR_ID_ROLE_DEPTNAME              32769 non-null object
MGR_ID_ROLE_FAMILY_DESC           32769 non-null object
MGR_ID_ROLE_CODE                  32769 non-null object
MGR_ID_ROLLUP_12                  32769 non-null obj

In [12]:
from category_encoders.target_encoder import TargetEncoder
from category_encoders.utils import *
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV,StratifiedKFold,KFold
from sklearn.metrics import *


def k_fold_target(train,test,columns,target,folds=10):
    kf=KFold()
    t=train
    v=test
    global_mean=t[target].mean()
    for col in columns:
        #t[col+'_enc']=np.nan
        #v[col+'_enc']=np.nan
        for i,j in kf.split(t):
            #tr,vl=t.iloc[i],t.iloc[j]
            m=t.loc[t.index[i]].groupby(col)[target].mean()
            t.loc[t.index[j],col+'_enc']=t.loc[t.index[j],col].map(m)
            #print(t[col+'_enc'].isnull().sum())
        t[col+'_enc']=t[col+'_enc'].fillna(global_mean)
        v[col+'_enc']=v[col].map(t.groupby(col)[target].mean())
        v[col+'_enc']=v[col+'_enc'].fillna(global_mean)
        #print(t[col+'_enc'].isnull().sum())
    t=t.drop(columns,axis=1)
    v=v.drop(columns,axis=1)

    return t,v
        

In [13]:
train,validate=train_test_split(d,test_size=0.3,random_state=123)

cols=list(d.columns)
target='target'
cols.remove('target')
t,v=k_fold_target(train,validate,cols,target)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [15]:
t.head()

,target,RESOURCE_enc,MGR_ID_enc,ROLE_DEPTNAME_enc,ROLE_FAMILY_DESC_enc,ROLE_CODE_enc,ROLLUP_12_enc,RESOURCE_MGR_ID_enc,RESOURCE_ROLE_DEPTNAME_enc,RESOURCE_ROLE_FAMILY_DESC_enc,...,MGR_ID_ROLE_DEPTNAME_enc,MGR_ID_ROLE_FAMILY_DESC_enc,MGR_ID_ROLE_CODE_enc,MGR_ID_ROLLUP_12_enc,ROLE_DEPTNAME_ROLE_FAMILY_DESC_enc,ROLE_DEPTNAME_ROLE_CODE_enc,ROLE_DEPTNAME_ROLLUP_12_enc,ROLE_FAMILY_DESC_ROLE_CODE_enc,ROLE_FAMILY_DESC_ROLLUP_12_enc,ROLE_CODE_ROLLUP_12_enc
7693,0,0.000000,0.363636,0.045161,0.064000,0.036282,0.030471,0.057808,0.000000,0.000000,...,0.363636,0.363636,0.000000,0.363636,0.098592,0.000000,0.046980,0.031812,0.059524,0.000000
17676,0,0.222222,0.000000,0.082927,0.033898,0.034404,0.136986,0.000000,0.333333,0.000000,...,0.000000,0.057808,0.000000,0.000000,0.024691,0.038835,0.148936,0.040268,0.166667,0.095238
1969,0,0.000000,0.000000,0.153846,0.033473,0.000000,0.037313,0.057808,0.057808,0.057808,...,0.000000,0.000000,0.000000,0.000000,0.571429,0.000000,0.000000,0.000000,0.000000,0.000000
15050,0,0.057808,0.000000,0.025641,0.000000,0.127907,0.030471,0.057808,0.057808,0.057808,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000
345,1,0.000000,0.000000,0.066667,0.400000,0.250000,0.111111,0.057808,0.000000,0.000000,...,0.000000,0.057808,0.057808,0.000000,0.400000,0.153846,0.066667,0.400000,0.400000,0.133333


In [16]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    return auc
    #print('AUC_ROC= ',auc)
    #print('Accuracy= ',acc)
    #print('confusion matrix:\n',confusion_matrix(y,pred))

In [17]:
def eliminate(algo,cols,xtr,ytr,xts,yts):
    mod=algo.fit(xtr[cols],ytr)
    auc=aucroc(mod,xts[cols],yts)
    
    high=0

    for i in tqdm(cols):
        c=cols.copy()
        c.remove(i)

        mod=algo.fit(xtr[c],ytr)
        auc_sc=aucroc(mod,xts[c],yts)
        if auc_sc>high:
            high=auc_sc
            r=i

    return high,r


In [20]:
def col_select(algo,step,xtr,ytr,xts,yts):
    
    
    cols=list(xtr.columns)
    p=[]
    removed=[]
    mod=algo.fit(xtr[cols],ytr)
    auc=aucroc(mod,xts[cols],yts)
    high_current=auc
    removed_new=[]
    high_new=auc
    best=auc
   # best_rem=[]
    while high_current-high_new<step:    


        removed=removed+removed_new
        print(len(removed))
        removed_new=[]


        if high_current==auc:
            pass
        else:
            high_current=high_new
        if len(cols)>2:
            high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
            while high_new>=high_current:
                cols.remove(r)
                removed_new.append(r)
                high_current=high_new

                high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
                print(high_new)
                if high_new>best:
                    best=high_new
                    best_rem=removed+removed_new+[r]
        else:
            break

    print('high_auc=',high_current)
    print('auc_one step ahead=',high_new)
    print('removed: ',removed)
    print('len=',len(removed))
    print('best=',best)
    print('best_rem=',best_rem)

In [21]:
xtr=t.drop(target,axis=1)
ytr=t[target]
xts=v.drop(target,axis=1)
yts=v[target]

In [22]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier()
col_select(lgb,0.1,xtr,ytr,xts,yts)

0


100%|██████████| 20/20 [00:09<00:00,  2.11it/s]


0.8828801541761073


100%|██████████| 19/19 [00:08<00:00,  2.14it/s]


0.8821430327605316
2


...

0.8559261914038121
17


100%|██████████| 3/3 [00:00<00:00,  3.21it/s]


0.8296928960219084
18


100%|██████████| 2/2 [00:00<00:00,  3.31it/s]


0.7955892810536628
19
high_auc= 0.7955892810536628
auc_one step ahead= 0.7955892810536628
removed:  ['ROLLUP_12_enc', 'ROLE_FAMILY_DESC_ROLLUP_12_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'ROLE_CODE_ROLLUP_12_enc', 'ROLE_DEPTNAME_enc', 'MGR_ID_ROLE_CODE_enc', 'ROLE_FAMILY_DESC_enc', 'RESOURCE_ROLE_FAMILY_DESC_enc', 'MGR_ID_ROLE_DEPTNAME_enc', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC_enc', 'RESOURCE_ROLE_DEPTNAME_enc', 'MGR_ID_enc', 'MGR_ID_ROLLUP_12_enc', 'RESOURCE_ROLE_CODE_enc', 'ROLE_DEPTNAME_ROLE_CODE_enc', 'RESOURCE_ROLLUP_12_enc', 'ROLE_CODE_enc', 'RESOURCE_MGR_ID_enc', 'ROLE_FAMILY_DESC_ROLE_CODE_enc']
len= 19
best= 0.8890362291156814
best_rem= ['ROLLUP_12_enc', 'ROLE_FAMILY_DESC_ROLLUP_12_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'ROLE_CODE_ROLLUP_12_enc', 'ROLE_DEPTNAME_enc', 'MGR_ID_ROLE_CODE_enc']
